# Install openclean
Using openclean to clean dataset.

In [ ]:
!pip install openclean[full]

In [1]:
%matplotlib inline
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.signal import savgol_filter
import csv
from openclean.data.load import dataset
from openclean.data.source.socrata import Socrata
from openclean.profiling.dataset import dataset_profile

# Apply the techniques you used for Part 1 and measure their effectiveness

In [41]:
# There are 1838945 rows in origial dataset and 1263421 rows after cleaned.
ori_data_cnt, cleaned_data_cnt = 1838945, 1263421

# remove invalid data type. e.g.
valid_data_cnt = 1263347

precision = cleaned_data_cnt / ori_data_cnt
recall = valid_data_cnt / cleaned_data_cnt

print("Original data cleaning precision is {}".format(precision))
print("Original data cleaning recall is {}".format(recall))

Original data cleaning precision is 0.6870357732286718
Original data cleaning recall is 0.9999414288665457


# Data cleaning target
Improve/refine your techniques to cover the new data and compare its effectiveness with your original approach.

We removed all invalid data include unaccepted data type and blank cell. 

To improve our cleaning stratagy, we are going to fill empty cell by 'Unspecified', convert address under naming convention, standardlizate all datetime fields.

Example，

Collision reason:
    Unattention,
    ...
    
Address: 
    Street: 181 East street
    City: New York
    State: NY
    Zipcode: 10010
    
Date: 2021-12-09 (yyyy-MM-DD)
Time: 16:44:00 (hh:mm:ss)


# Create reference data for the data types you cleaned.


In [16]:
# Add zipcode reference

nyc_borough_zipcode_ref = pd.read_csv('data/ZIP_BOROUGH.csv', dtype=str)

nyc_zipcode_ref = {
    'Staten Islant':[10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10311,10312,\
                      10314,10301,10302,10303,10304,10305,10306,10307,10308,10309,10310,10311,\
                      10312,10314],
    'Queens':[11004,11101,11102,11103,11104,11105,11106,11109,11351,11354,11355,11356,11357,11358,\
              11359,11360,11361,11362,11363,11364,11365,11366,11367,11368,11369,11370,11371,11372,\
              11373,11374,11375,11377,11378,11379,11385,11411,11412,11413,11414,11415,11416,11417,\
              11418,11419,11420,11421,11422,11423,11426,11427,11428,11429,11430,11432,11433,11434,\
              11435,11436,11691,11692,11693,11694,11697,11101,11102,11103,11004,11104,11105,11106,\
              11109,11351,11354,11355,11356,11357,11358,11359,11360,11361,11362,11363,11364,11365,\
              11366,11367,11368,11369,11370,11371,11372,11373,11374,11375,11377,11378,11379,11385,\
              11411,11412,11413,11414,11415,11416,11417,11418,11419,11420,11421,11422,11423,11426,\
              11427,11428,11429,11430,11432,11433,11434,11435,11436,11691,11692,11693,11694,11697], 
    'Manhattan':[10001,10002,10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10015,\
                 10016,10017,10018,10019,10020,10021,10022,10023,10024,10025,10026,10027,10028,10029,\
                 10030,10031,10032,10033,10034,10035,10036,10037,10038,10039,10040,10041,10044,10045,\
                 10048,10055,10060,10069,10090,10095,10098,10099,10103,10104,10105,10106,10107,10110,\
                 10111,10112,10115,10118,10119,10120,10121,10122,10123,10128,10151,10152,10153,10154,\
                 10155,10158,10161,10162,10165,10166,10167,10168,10169,10170,10171,10172,10173,10174,\
                 10175,10176,10177,10178,10199,10270,10271,10278,10279,10280,10281,10282,10001,10002,\
                 10003,10004,10005,10006,10007,10009,10010,10011,10012,10013,10014,10015,10016,10017,\
                 10018,10019,10020,10021,10022,10023,10024,10025,10026,10027,10028,10029,10030,10031,\
                 10032,10033,10034,10035,10036,10037,10038,10039,10040,10041,10044,10045,10048,10055,\
                 10060,10069,10090,10095,10098,10099,10103,10104,10105,10106,10107,10110,10111,10112,\
                 10115,10118,10119,10120,10121,10122,10123,10128,10151,10152,10153,10154,10155,10158,\
                 10161,10162,10165,10166,10167,10168,10169,10170,10171,10172,10173,10174,10175,10176,\
                 10177,10178,10199,10270,10271,10278,10279,10280,10281,10282],
    'Bronx':[10451,10452,10453,10454,10455,10456,10457,10458,10459,10460,10461,10462,10463,10464,10465,\
             10466,10467,10468,10469,10470,10471,10472,10473,10474,10475,10451,10452,10453,10454,10455,\
             10456,10457,10458,10459,10460,10461,10462,10463,10464,10465,10466,10467,10468,10469,10470,\
             10471,10472,10473,10474,10475]
    }

print(nyc_zipcode_ref)

{'Staten Islant': [10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311, 10312, 10314, 10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 10311, 10312, 10314], 'Queens': [11004, 11101, 11102, 11103, 11104, 11105, 11106, 11109, 11351, 11354, 11355, 11356, 11357, 11358, 11359, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428, 11429, 11430, 11432, 11433, 11434, 11435, 11436, 11691, 11692, 11693, 11694, 11697, 11101, 11102, 11103, 11004, 11104, 11105, 11106, 11109, 11351, 11354, 11355, 11356, 11357, 11358, 11359, 11360, 11361, 11362, 11363, 11364, 11365, 11366, 11367, 11368, 11369, 11370, 11371, 11372, 11373, 11374, 11375, 11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 114

## Cleaning column: BOROUGH    


Source dataset:
https://data.cityofnewyork.us/Transportation/Automated-Traffic-Volume-Counts/7ym2-wayt

Column name: Boro

Origial:
Delete it if it was empty or invalid Borough

Improvement:
Fill 'Unspecified' category

In [17]:
# sample data
# datafile = "data/Automated_Traffic_Volume_Counts_sample1.csv"

# original data
datafile = "data/Automated_Traffic_Volume_Counts.csv"
# load the file in memory
ds = dataset(datafile)


In [18]:
from openclean.function.value.null import is_empty
from openclean.operator.transform.update import update

ds = update(ds, ["Boro"], lambda x: "Unspecified" if is_empty(x) else x)

In [19]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
RequestID,27190511,0,7090,2.607527e-04,10.870351
Boro,27190511,0,5,1.838877e-07,2.174944
Yr,27190511,0,16,5.884406e-07,3.447653
M,27190511,0,12,4.413304e-07,3.334093
D,27190511,0,31,1.140104e-06,4.925571
HH,27190511,0,24,8.826609e-07,4.584959
MM,27190511,0,4,1.471101e-07,1.999995
Vol,27190511,0,4016,1.476986e-04,8.072537
SegmentID,27190511,0,14953,5.499345e-04,13.164284
WktGeom,27190511,0,20463,7.525787e-04,13.880069


## Cleaning column: BOROUGH 

Source dataset:
https://data.cityofnewyork.us/Transportation/Traffic-Signal-and-All-Way-Stop-Study-Requests/w76s-c5u4

Column name: Borough

Origial:
Delete it if it was empty or invalid Borough

Improvement:
Fill 'Unspecified' category

In [46]:
# sample data
# datafile = "data/Traffic_Signal_and_All-Way_Stop_Study_Requests_sample1.csv"

# original data
datafile = "data/Traffic_Signal_and_All-Way_Stop_Study_Requests.csv"
# load the file in memory
ds = dataset(datafile, encoding="utf-8")


In [47]:
from openclean.function.value.null import is_empty
from openclean.operator.transform.update import update

ds = update(ds, ["Borough"], lambda x: "Unspecified" if is_empty(x) else x)

In [48]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Id,60130,0,60044,0.998570,15.872474
ReferenceNumber,60130,0,60044,0.998570,15.872474
StudyUnit,60130,0,2,0.000033,0.874897
DateCreated,60130,39225,1542,0.073762,10.206735
OldReferenceNumber,60130,43180,16364,0.965428,13.884466
ExternalReferenceNumber,60130,12933,42425,0.898892,15.196084
DateRequested,60130,0,5856,0.097389,11.972782
StatusDescription,60130,0,57,0.000948,2.268256
StudyStatus,60130,0,1,0.000017,0.000000
StatusDate,60130,283,6304,0.105335,12.123976


## Cleaning column: BOROUGH/ZIPCODE

Source dataset:
https://data.cityofnewyork.us/City-Government/Mobile-Telecommunications-Franchise-Pole-Reservati/tbgj-tdd6

Column name: Borough/Zipcode

Origial:
Delete it if it was empty or invalid Borough

Improvement:
Based on zipcode fill Borough field if this cell was empty

In [21]:
def find_key(input_dict, value):
    return next((k for k, v in input_dict.items() if v == value), None)

In [44]:
# sample data
# datafile = "data/Mobile_Telecommunications_Franchise_Pole_Reservation_Locations_sample1.csv"

# original data
datafile = "data/Mobile_Telecommunications_Franchise_Pole_Reservation_Locations.csv"
# load the file in memory
ds = dataset(datafile, encoding="utf-8")


ds = update(ds, ["Borough", "Zipcode"], lambda x, y: ("Unspecified", y) if is_empty(x) and is_empty(y) else (x, y))
ds = update(ds, ["Borough", "Zipcode"], lambda x, y: (x, y) if not is_empty(x) else (find_key(nyc_zipcode_ref, y), y))

In [45]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Id,13388,0,13388,1.000000,13.708653
Reservation Date,13388,0,444,0.033164,7.241078
Franchisee Name,13388,0,12,0.000896,3.059333
Status,13388,0,3,0.000224,1.438882
Installation Date,13388,6244,1287,0.180151,9.520133
Pole Class,13388,0,2,0.000149,0.883887
Borough,13388,0,5,0.000373,2.236662
X Coord.,13388,0,12495,0.933298,13.572002
Y Coord.,13388,0,12791,0.955408,13.618284
Latitude,13388,0,13347,0.996938,13.702528


## Cleaning column: CONTRIBUTING FACTOR VEHICLE 1(Collision reason)

Source dataset:
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu

Column name: Pedestrain - CONTRIBUTING_FACTOR_1

Origial:
Delete it if it was empty or invalid CONTRIBUTING_FACTOR

Improvement:
Using KNN to find most similar reason cause the collision

In [23]:
from openclean.cluster.knn import knn_clusters
from openclean.function.similarity.base import SimilarityConstraint
from openclean.function.similarity.text import LevenshteinDistance
from openclean.function.value.threshold import GreaterThan
from openclean.function.eval.domain import Lookup


# sample data
# datafile = "data/Person_sample11.csv"

# original data
datafile = "data/Motor_Vehicle_Collisions_-_Person.csv"
# load the file in memory
ds = dataset(datafile)


# cleaning using cluster and mapping
for i in range(1, 3): 
    col_name = "CONTRIBUTING_FACTOR_{}".format(i)

    # edit distance cluster
    clusters = knn_clusters(values=ds[col_name].unique().tolist(),\
                            sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.8))
          )

    mapping = {}
    for cluster in clusters: 
        mapping.update(cluster.to_mapping())

    ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [24]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
UNIQUE_ID,4523558,0,4523558,1.000000e+00,22.109027
COLLISION_ID,4523558,0,1219446,2.695767e-01,19.998231
CRASH_DATE,4523558,0,3446,7.617897e-04,11.149194
CRASH_TIME,4523558,0,1440,3.183335e-04,8.980054
PERSON_ID,4523558,19,4328735,9.569355e-01,21.407822
PERSON_TYPE,4523558,0,4,8.842597e-07,0.255635
PERSON_INJURY,4523558,0,3,6.631948e-07,0.522585
VEHICLE_ID,4523558,178029,2077696,4.781227e-01,20.841487
PERSON_AGE,4523558,420524,849,2.069200e-04,5.825266
EJECTION,4523558,2209750,6,2.593128e-06,0.157668


## Cleaning column: CONTRIBUTING FACTOR VEHICLE 1(Collision reason)

Source dataset:
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Vehicles/bm4k-52h4

Column name: Vechicles - CONTRIBUTING_FACTOR_1

Origial:
Delete it if it was empty or invalid CONTRIBUTING_FACTOR

Improvement:
Using KNN to find most similar reason cause the collision

In [49]:
# sample data
# datafile = "data/Motor_Vehicle_Collisions_-_Vehicles_sample1.csv"

# original data
datafile = "data/Motor_Vehicle_Collisions_-_Vehicles.csv"
# load the file in memory
ds = dataset(datafile)


# cleaning using cluster and mapping
for i in range(1, 3): 
    col_name = "CONTRIBUTING_FACTOR_{}".format(i)

    # edit distance cluster
    clusters = knn_clusters(values=ds[col_name].unique().tolist(),\
                            sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.8))
          )

    mapping = {}
    for cluster in clusters: 
        mapping.update(cluster.to_mapping())

    ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [26]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
UNIQUE_ID,3704406,0,3704406,1.000000,21.820811
COLLISION_ID,3704406,0,1845435,0.498173,20.773098
CRASH_DATE,3704406,0,3444,0.000930,11.688995
CRASH_TIME,3704406,0,1440,0.000389,8.932319
VEHICLE_ID,3704406,0,2175578,0.587295,13.859491
STATE_REGISTRATION,3704406,217048,82,0.000024,1.233480
VEHICLE_TYPE,3704406,177774,2145,0.000608,3.402818
VEHICLE_MAKE,3704406,1785296,10271,0.005352,4.860344
VEHICLE_MODEL,3704406,3652982,2432,0.047293,7.900430
VEHICLE_YEAR,3704406,1796971,299,0.000157,4.376369


## Cleaning column: VEHICLE_OCCUPANTS

Source dataset:
https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Person/f55k-p6yu

Column name: Vehicles - ["VEHICLE_OCCUPANTS"]

Origial:
Delete it if it was empty or invalid ["VEHICLE_OCCUPANTS"]

Improvement:
fill 0 if these columns are empty

In [50]:
ds = update(ds, ["VEHICLE_OCCUPANTS"], lambda x: 0 if is_empty(x) else x)

In [ ]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

## Cleaning column: VEHICLE TYPE CODE(Vechicle type)

Source dataset:
https://data.cityofnewyork.us/Transportation/Vehicle-Classification-Counts-2014-2019-/96ay-ea4r

Column name: Veh Class Type

Origial:
Delete it if it was empty or invalid Veh Class Type

Improvement:
Using KNN to find most similar vehicle type

In [27]:
# sample data
# datafile = "data/Vehicle_Classification_Counts_sample1.csv"

# original data
datafile = "data/Vehicle_Classification_Counts__2014-2019_.csv"
# load the file in memory
ds = dataset(datafile)

# cleaning letter case and null

col_name = "Veh Class Type"
ds = update(ds, col_name, str.title)

ds = update(ds, col_name, lambda x: "Unknown" if is_empty(x) else x)

In [28]:
from openclean.cluster.knn import knn_collision_clusters
from openclean.function.similarity.text import JaroWinklerSimilarity

# cleaning using cluster and mapping

col_name = "Veh Class Type"

# edit distance cluster
clusters = knn_collision_clusters(values=ds[col_name].tolist(),\
                                  sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.7))
     )

mapping = {}
for cluster in clusters: 
    mapping.update(cluster.to_mapping())

ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

# common substr cluster
clusters = knn_collision_clusters(values=ds[col_name].tolist(),\
                                  sim=SimilarityConstraint(func=JaroWinklerSimilarity(), pred=GreaterThan(0.9))
     )

mapping = {}
for cluster in clusters: 
    mapping.update(cluster.to_mapping())

ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [29]:
# select top 20 types and only keep these types
# all other types are regarded as unrecognizable values
top_types = ds["Veh Class Type"].value_counts().head(20).keys()

col_name = "Veh Class Type"

ds = update(ds, col_name, lambda x: "Unknown" if x not in top_types else x)

In [30]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
ID,10304,0,120,0.011646,6.861351
SegmentID,10304,0,645,0.062597,9.142148
Roadway Name,10304,0,444,0.043090,8.438977
From,10304,0,499,0.048428,8.745615
To,10304,0,500,0.048525,8.766482
Direction,10304,0,5,0.000485,2.004288
Date,10304,0,155,0.015043,6.699855
Veh Class Type,10304,0,10,0.000970,3.067400
12:00-1:00 AM,10304,3276,441,0.062749,4.556366
1:00-2:00AM,10304,3276,347,0.049374,4.237499


## Cleaning column: CRASH DATE/CRASH TIME

Source dataset:
https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/pvqr-7yc4

Column name: Vehicle Expiration Date

Origial:
Delete it if it was empty or invalid Date

Improvement:
Convert invalid data type to datetime

In [31]:
# sample data
# df = pd.read_csv('data/Parking_Violations_Issued_-_Fiscal_Year_2022_sample1.csv', dtype=str)

# original data
df = pd.read_csv('data/Parking_Violations_Issued_-_Fiscal_Year_2022.csv', dtype=str)
df['Vehicle Expiration Date'] = pd.to_datetime(df['Vehicle Expiration Date'], format='%y%m%d', errors='coerce')


In [32]:
df

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1457617912,JEB5683,NY,PAS,06/25/2021,40,VAN,FORD,P,63430,...,GY,0,2007,-,3,NaN,NaN,NaN,NaN,NaN
1,1457617924,JAN2986,NY,PAS,06/25/2021,20,SUBN,DODGE,P,13490,...,BLU,0,2007,-,0,NaN,NaN,NaN,NaN,NaN
2,1457622427,FJH6630,TX,PAS,06/17/2021,98,SDN,AUDI,P,79430,...,WHITE,0,0,-,0,NaN,NaN,NaN,NaN,NaN
3,1457638629,RD1Y5N,MO,PAS,06/16/2021,98,SDN,TOYOT,P,53130,...,TAN,0,2001,-,0,NaN,NaN,NaN,NaN,NaN
4,1457639580,T503814C,NY,OMT,07/04/2021,40,TAXI,HONDA,P,81030,...,WHI,0,2020,-,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5346912,8996899756,KDW1332,NY,PAS,10/03/2021,14,SUBN,NISSA,T,54000,...,BK,NaN,2014,NaN,0,T,14-No Standing,NaN,NaN,NaN
5346913,8996899768,KDW1332,NY,PAS,10/03/2021,71,SUBN,NISSA,T,54000,...,BK,NaN,2014,NaN,0,T,71A-Insp Sticker Expired (NYS),NaN,NaN,NaN
5346914,8996899770,KMA3890,NY,OMS,10/03/2021,14,4DSD,HYUND,T,78830,...,BK,NaN,2021,NaN,0,T,14-No Standing,NaN,NaN,NaN
5346915,8996899781,KLT7668,NY,PAS,10/03/2021,14,4DSD,VOLKS,T,54000,...,RD,NaN,2016,NaN,0,T,14-No Standing,NaN,NaN,NaN


## Cleaning column: ON STREET NAME

Source dataset:
https://data.cityofnewyork.us/Transportation/Street-Closures-due-to-construction-activities-by-/i6b5-j7bu

Column name: ONSTREETNAME

Origial:
Delete it if it was empty or invalid ONSTREETNAME

Improvement:
Using KNN to find most similar Street name

In [33]:
# sample data
# datafile = "data/Traffic_Signal_and_All-Way_Stop_Study_Requests.csv"

# original data
datafile = "data/Street_Closures_due_to_construction_activities_by_Block_sample1.csv"
# load the file in memory
ds = dataset(datafile)

col_name = "ONSTREETNAME"

# edit distance cluster
clusters = knn_clusters(values=ds[col_name].unique().tolist(),\
                        sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.8))
      )

mapping = {}
for cluster in clusters: 
    mapping.update(cluster.to_mapping())

ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [34]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
SEGMENTID,107475,0,33338,0.310193,14.124876
ONSTREETNAME,107475,0,2245,0.020889,8.568400
FROMSTREETNAME,107475,5,4287,0.039890,9.811557
TOSTREETNAME,107475,5,4525,0.042105,10.188625
BOROUGH_CODE,107475,0,5,0.000047,2.245762
WORK_START_DATE,107475,0,24197,0.225141,12.441400
WORK_END_DATE,107475,0,2437,0.022675,10.261630
PURPOSE,107475,0,102,0.000949,3.353200


## Cleaning column: ON STREET NAME

Source dataset:
https://data.cityofnewyork.us/Transportation/Street-Closures-due-to-construction-activities-by-/478a-yykk

Column name: ONSTREETNAME

Origial:
Delete it if it was empty or invalid ONSTREETNAME

Improvement:
Using KNN to find most similar Street name

In [35]:
# sample data
# datafile = "data/Street_Closures_due_to_construction_activities_by_Intersection_sample1.csv"

# original data
datafile = "data/Street_Closures_due_to_construction_activities_by_Intersection.csv"
# load the file in memory
ds = dataset(datafile)

col_name = "ONSTREETNAME"

# edit distance cluster
clusters = knn_clusters(values=ds[col_name].unique().tolist(),\
                        sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.8))
      )

mapping = {}
for cluster in clusters: 
    mapping.update(cluster.to_mapping())

ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [36]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
NODEID,28609,0,13387,0.467930,13.452021
ONSTREETNAME,28609,0,1496,0.052291,8.051996
FROMSTREETNAME,28609,0,2734,0.095564,10.421487
BOROUGH_CODE,28609,0,5,0.000175,1.866993
WORK_START_DATE,28609,0,13286,0.464399,12.240667
WORK_END_DATE,28609,0,1171,0.040931,8.354404
PURPOSE,28609,0,56,0.001957,1.908148


## Cleaning column: ON STREET NAME

Source dataset:
https://data.cityofnewyork.us/Transportation/Parking-Regulation-Locations-and-Signs/xswq-wnv9

Column name: main_st

Origial:
Delete it if it was empty or invalid main_st

Improvement:
Using KNN to find most similar Street name

In [37]:
# sample data
# datafile = "data/locations_sample1.csv"

# original data
datafile = "data/locations.csv"
# load the file in memory
ds = dataset(datafile)

col_name = "main_st"

# edit distance cluster
clusters = knn_clusters(values=ds[col_name].unique().tolist(),\
                        sim=SimilarityConstraint(func=LevenshteinDistance(), pred=GreaterThan(0.7))
      )

mapping = {}
for cluster in clusters: 
    mapping.update(cluster.to_mapping())

ds = update(ds, col_name, Lookup(columns=[col_name], mapping=mapping, default=col_name))

In [38]:
# review dataset profile
profiles = dataset_profile(ds)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
boro,96030,0,5,0.000052,2.123173
order_no,96030,0,96030,1.000000,16.551198
main_st,96030,0,1990,0.020723,8.572809
from_st,96030,0,7590,0.079038,11.340783
to_st,96030,0,7639,0.079548,11.344984
sos,96030,0,5,0.000052,2.000139
